# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

.fetchone()
.fetchmany()
.fetchall()
.executemany()
.executescript()

In [1]:
import csv
# import json
# import pickle
# import pandas as pd
# import numpy as np
# import h5py

# from bs4 import BeautifulSoup as bs
from pprint import pprint as pp
import sqlite3

In [ ]:
con = sqlite3.connect('./data/data/Chinook_Sqlite.sqlite')
cur = con.cursor()


cur.execute("""
SELECT firstname, lastname
FROM Customer
WHERE country = ? AND state = ?
""", ('Canada', 'QC'))

cur.close()
con.close()

In [ ]:
con = sqlite3.connect('./data/data/test.db')
cur = con.cursor()


cur.execute("""
CREATE TABLE Student(id INTEGER, name VARCHAR(50))
""")

cur.execute("""
INSERT INTO Student(id, name) VALUES (1, 'Daniil')
""")
con.commit()

cur.close()
con.close()

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

2. Найти и вывести на экран названия всех альбомов группы Accept

3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор.

In [ ]:
conn = sqlite3.connect('./data/data/recipes.db')
curr = conn.cursor()


curr.close()
conn.close()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [3]:
conn = sqlite3.connect('./data/data/recipes.db')
curr = conn.cursor()

curr.execute("""
CREATE TABLE IF NOT EXISTS Recipe(
	id              INTEGER PRIMARY KEY,
	name            VARCHAR(80),
	minutes         TIME,
	submitted       DATE,
	description     TEXT,
	n_ingredients   INTEGER
);
""")

curr.close()
conn.close()

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [9]:
conn = sqlite3.connect('./data/data/recipes.db')
curr = conn.cursor()

curr.execute("""
CREATE TABLE IF NOT EXISTS Review(
	id          INTEGER PRIMARY KEY,
	user_id     INTEGER,
	recipe_id   INTEGER,
	date        DATE,
	rating      INTEGER,
	review      TEXT,
	FOREIGN KEY (
		recipe_id
	)
	REFERENCES Recipe(id)
);
""")

curr.close()
conn.close()

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [82]:
conn = sqlite3.connect('./data/data/recipes.db')
curr = conn.cursor()

with open("./data/data/recipes_sample_with_tags_ingredients.csv", "r", encoding="UTF-8") as recipes_file:
	dr = csv.DictReader(recipes_file)
	values_to_db = [(i["id"], i["name"], i["minutes"], i["submitted"], i["description"], i["n_ingredients"]) for i in dr]
	query="""
	INSERT INTO Recipe (id, name, minutes, submitted, description, n_ingredients)
	VALUES (?, ?, ?, ?, ?, ?);"""
	# curr.executemany(query, values_to_db)
	# conn.commit()

with open("./data/data/reviews_sample.csv", "r", encoding="UTF-8") as reviews_file:
	dr = csv.DictReader(reviews_file)
	values_to_db = [(i[""], i["user_id"], i["recipe_id"], i["date"], i["rating"], i["review"]) for i in dr]
	query = """
	INSERT INTO Review (id, user_id, recipe_id, date, rating, review)
	VALUES (?, ?, ?, ?, ?, ?);"""
	# curr.executemany(query, values_to_db)
	# conn.commit()

curr.close()
conn.close()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [98]:
conn = sqlite3.connect('./data/data/recipes.db')
curr = conn.cursor()

query = """
SELECT * FROM Recipe
	WHERE n_ingredients=10;"""

curr.execute(query)
records = curr.fetchmany(5)

print(records)

curr.close()
conn.close()

[(176, 'cola cake', 55, '1999-08-22', '', 10), (246, 'lee s hot crab dip', 45, '1999-09-01', "lee's hot crab dip", 10), (289, 'feijoada  brazilian bean soup  ii', 150, '1999-10-04', 'brazilian bean soup', 10), (373, 'tiramisu   balducci s in new york city', 95, '1999-08-17', 'this is an adopted recipe. i shall be trying it out and then post any additional information if needs be. if you are going to give any or no stars, please at least give the reason why so that you can help me improve the recipe. thank you.', 10), (544, 'greek mushroom salad', 38, '1999-08-10', '', 10)]


6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [106]:
conn = sqlite3.connect('./data/data/recipes.db')
curr = conn.cursor()

query = """
SELECT *
	FROM Recipe
	WHERE minutes = (SELECT MAX(minutes) FROM Recipe);"""

curr.execute(query)
records = curr.fetchone()

print(records)

curr.close()
conn.close()

(236274, 'strawberry liqueur', 129615, '2007-06-21', 'a beautiful light red shade with a delicious strawberry flavor.', 5)


7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [158]:
conn = sqlite3.connect('./data/data/recipes.db')
curr = conn.cursor()

given_id = (44123, )

query = """
SELECT *
	FROM Recipe
	WHERE id = ?;"""

curr.execute(query, given_id)
records = curr.fetchone()

if records:
	print(records)
else:
	print("Рецепта с заданным id не найдено!")

curr.close()
conn.close()

(44123, 'george s at the cove  black bean soup', 90, '2002-10-25', "an original recipe created by chef scott meskan, george's at the cove. we enjoyed this when we visited this restaurant in la jolla, california. this recipe is requested so often, they have it printed and ready at the hostess stand. it's unbeatable at the restaurant, but i do a pretty good job at home, too, if i do say so myself!", 18)


8. Найдите кол-во отзывов с рейтингом 5.

In [124]:
conn = sqlite3.connect('./data/data/recipes.db')
curr = conn.cursor()

query = """
SELECT COUNT(*)
	FROM Review
	WHERE rating = 5;"""

curr.execute(query)
records = curr.fetchall()

print(records[0][0])

curr.close()
conn.close()

91361


9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4.

In [40]:
conn = sqlite3.connect('./data/data/recipes.db')
curr = conn.cursor()

query = """
SELECT COUNT(*)
	FROM Review
	WHERE rating >= 4 AND review = ''
	GROUP BY recipe_id;
"""

curr.execute(query)
records = curr.fetchall()

pp(len(records))

curr.close()
conn.close()

15


10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [4]:
conn = sqlite3.connect('./data/data/recipes.db')
curr = conn.cursor()

query = """
SELECT COUNT(*)
	FROM Recipe
	WHERE (submitted BETWEEN '2010-01-01' AND '2010-12-31') AND minutes >= 15;"""

curr.execute(query)
records = curr.fetchall()

print(records[0][0])

curr.close()
conn.close()

1319


11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [38]:
conn = sqlite3.connect('./data/data/recipes.db')
curr = conn.cursor()

query = """
SELECT REV.recipe_id, REC.name, REV.user_id, REV.date, REV.rating
	FROM Review REV INNER JOIN Recipe REC on REC.id = REV.recipe_id
	WHERE n_ingredients >= 3
	ORDER BY REC.id;"""

curr.execute(query)
records = curr.fetchmany(20)

pp(records)

curr.close()
conn.close()


[(48, 'boston cream pie', 32421, '2002-03-15', 0),
 (48, 'boston cream pie', 68674, '2004-05-03', 2),
 (55, 'betty crocker s southwestern guacamole dip', 53959, '2006-01-12', 4),
 (55, 'betty crocker s southwestern guacamole dip', 165567, '2006-03-31', 5),
 (55, 'betty crocker s southwestern guacamole dip', 1060485, '2009-04-07', 5),
 (55, 'betty crocker s southwestern guacamole dip', 851190, '2010-05-23', 5),
 (66, 'black coffee barbecue sauce', 42938, '2002-10-21', 4),
 (66, 'black coffee barbecue sauce', 8679, '2003-03-17', 5),
 (66, 'black coffee barbecue sauce', 136813, '2007-04-14', 5),
 (66, 'black coffee barbecue sauce', 498631, '2007-05-11', 5),
 (66, 'black coffee barbecue sauce', 352271, '2007-11-05', 5),
 (66, 'black coffee barbecue sauce', 314792, '2008-02-04', 5),
 (66, 'black coffee barbecue sauce', 136004, '2008-06-05', 5),
 (66, 'black coffee barbecue sauce', 330545, '2009-01-02', 5),
 (66, 'black coffee barbecue sauce', 675265, '2009-01-10', 5),
 (66, 'black coffee ba